### Regression notebook for Wadhwani AI competition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import timm
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2,torchvision
from ipyexperiments.ipyexperiments import IPyExperimentsPytorch
from timm.optim.optim_factory import create_optimizer_v2
from timm import utils
from fastprogress.fastprogress import format_time
from fastai.vision.all import *


class CFG:
    seed = 46
    n_splits = 5
    SZ = 1024
    debug = False
    BS = 12
    EP = 10
    MODEL = 'tf_efficientnet_b0_ns'
    LR = 5e-03
    WD = 1e-08
    debug = False

random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)
plt.rcParams["font.size"] = 13
warnings.filterwarnings('ignore')

In [2]:
set_seed(CFG.seed)

In [3]:
DIR = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
IMG_PATH = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images'
submit = pd.read_csv(os.path.join(DIR,'SampleSubmission.csv'))
train = pd.read_csv(os.path.join(DIR,'Train.csv'))
test_df = pd.read_csv(os.path.join(DIR,'Test.csv'))

if CFG.debug:
    train = train.sample(frac=0.1).reset_index(drop=True)
    
VERSION = "NB_EXP_V0_002_Classification"
MODEL_FOLDER = Path(f"///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/Classification/{VERSION}/")
os.makedirs(MODEL_FOLDER,exist_ok=True)
KERNEL_TYPE = f"{CFG.MODEL}_{CFG.SZ}_bs{CFG.BS}_ep{CFG.EP}_lr{str(CFG.LR).replace('-','')}_wd{str(CFG.WD).replace('-','')}"

print(MODEL_FOLDER)
print(KERNEL_TYPE)

/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/Classification/NB_EXP_V0_002_Classification
tf_efficientnet_b0_ns_1024_bs12_ep10_lr0.005_wd1e08


In [4]:
def make_train_dataset():
    train_new_pbw = pd.DataFrame()
    train_new_pbw['image_id_worm']= train['image_id_worm'].unique()
    train_new_pbw = pd.merge(train_new_pbw,train[train['worm_type']=='pbw'].reset_index(drop=True),on='image_id_worm',how='left')
    train_new_pbw['worm_type'] = 'pbw'
    train_new_pbw.fillna(0,inplace=True)
    
    train_new_abw = pd.DataFrame()
    train_new_abw['image_id_worm']= train['image_id_worm'].unique()
    train_new_abw = pd.merge(train_new_abw,train[train['worm_type']=='abw'].reset_index(drop=True),on='image_id_worm',how='left')
    train_new_abw['worm_type'] = 'abw'
    train_new_abw.fillna(0,inplace=True)
    
    train_out = pd.concat([train_new_pbw,train_new_abw],0).reset_index(drop=True)
    
    assert len(train_out) == train['image_id_worm'].nunique()*2
    train_out = pd.pivot(train_out,'image_id_worm','worm_type','number_of_worms').reset_index()
    train_out[['abw','pbw']] = train_out[['abw','pbw']].astype(int)
    
    train_out['abw_cls'] = (train_out['abw']>0).astype(int)
    train_out['pbw_cls'] = (train_out['pbw']>0).astype(int)
    
    return train_out

train_new = make_train_dataset()

In [5]:
train_new.head(1)

worm_type,image_id_worm,abw,pbw,abw_cls,pbw_cls
0,id_0002ea6f15c7fa6f4c221783.jpg,0,51,0,1


In [19]:
train_new['abw'].value_counts()[:25]

0     8042
5      253
1      243
6      180
3      171
4      160
7      152
8      147
2      106
9      100
10      72
11      39
12      29
15      12
13      12
14      10
16       7
22       1
23       1
Name: abw, dtype: int64

In [26]:
(train_new['pbw']>200).sum()

69

### Get kfolds

In [ ]:
train_files = []
val_files = []
fold = []

for folds in [0,1,2,3,4]:
    files = list(pd.read_csv(f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/splits/fold{folds}.txt',header=None)[0].apply(lambda x:x.split("/")[-1]))
    fold.append([folds]*len(files))
    train_files.append(files)

train_files= ([item for sublist in train_files for item in sublist])
fold= ([item for sublist in fold for item in sublist])
fold_dict = dict(zip(train_files,fold))
train_new['fold'] = train_new['image_id_worm'].map(fold_dict)
train_new = train_new.loc[~train_new['fold'].isna()].reset_index(drop=True)

In [ ]:
train_new.head()

In [ ]:
test_df.head(1)

#### Data loader

In [ ]:
class WadhwaniDataset(Dataset):
    def __init__(self,
                 df=train_new,
                 mode='train',
                augs = None):
        
        self.augs = augs
        self.df = df
        self.mode  = mode
        
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, ix):
        image_id = self.df.loc[ix,'image_id_worm']
        img_path = f'{IMG_PATH}/{image_id}'
        
        img = Image.open(img_path).convert("RGB")
        img = np.array(img)
        
        if self.augs is not None:
            img = self.augs(image=img)['image']
    
        if self.mode == 'test':
            return img
        
        label = torch.tensor(self.df[['abw_cls','pbw_cls']].loc[ix]).float()
        return img, label

In [ ]:
def worker_init_fn(worker_id):
    """
    Handles PyTorch x Numpy seeding issues.
    Args:
        worker_id (int): Id of the worker.
    """
    np.random.seed(np.random.get_state()[1][0] + worker_id)

### Augmentations

In [ ]:

TRAIN_AUG = A.Compose([
#     A.RandomResizedCrop(height=SZ, width=SZ, p=1.0, scale=(0.72, 1.0)),
#     A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(rotate_limit=45, border_mode=0, p=0.75),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
    A.Blur(p=0.5),
#     A.CoarseDropout(max_height=int(32*(CFG.SZ/512)), max_width=int(32*(CFG.SZ/512)), p=0.75),
    A.Resize(CFG.SZ,CFG.SZ,p=1),
    A.Normalize(),
    ToTensorV2(),
])

VALID_AUG = A.Compose([
#     A.SmallestMaxSize(max_size=CFG.SZ + 16, p=1.0),
#     A.CenterCrop(height=CFG.SZ, width=CFG.SZ, p=1.0),
    A.Resize(CFG.SZ,CFG.SZ,p=1),
    A.Normalize(),
    ToTensorV2(),
])

### Visualization

In [ ]:
dataset_show = WadhwaniDataset(train_new, augs=TRAIN_AUG, mode='train')
loader_show = torch.utils.data.DataLoader(dataset_show, batch_size=6)
img,target = next(iter(loader_show))

grid = torchvision.utils.make_grid(img, normalize=True, padding=2)
grid = grid.permute(1, 2, 0)
show_image(grid, figsize=(15,8),title=[x for x in target.numpy()]);

### Model

In [ ]:
def get_wadhwani_classification_model(model_name, pretrained=True, **kwargs):
    model = timm.create_model(model_name, pretrained=pretrained, **kwargs)
    model = nn.Sequential(model, nn.Dropout(0.15), nn.Linear(model.num_classes, 2))
    return model

In [ ]:
dl = DataLoader(WadhwaniDataset(train_new, augs=TRAIN_AUG, mode='train'),
                          batch_size=2,
                          shuffle=True,
                          num_workers=8,
                          drop_last=True,
                        worker_init_fn=worker_init_fn)

a,b = next(iter(dl))
a.shape,b[0].shape,b[1].shape

In [ ]:
m = get_wadhwani_classification_model(CFG.MODEL)
out = m(a)
print(out, out.shape)

In [ ]:
 nn.BCEWithLogitsLoss()(out,b)

### Train & Validation Function

In [ ]:
# class CombinedL1Loss:
#     "Dice and Focal combined"
#     def __init__(self, axis=1, smooth=1., alpha=0.5):
#         store_attr()
#         self.loss1 = torch.nn.L1Loss()
#         self.loss2 =  torch.nn.L1Loss()
        
#     def __call__(self, pred, targ):
#         loss = self.loss1(pred[0], targ[0]) + self.dice_loss(pred[1], targ[1])
#         return loss
    
#     def decodes(self, x):    return x.argmax(dim=self.axis)
#     def activation(self, x): return F.softmax(x, dim=self.axis)

In [ ]:
def train_one_epoch(
    model: nn.Module,
    loader: Iterable,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    lr_scheduler: torch.optim.lr_scheduler._LRScheduler = None,
    mixup_fn: Callable = None,
    grad_scaler: torch.cuda.amp.GradScaler = None,
    mbar: master_bar = None,
):

    model.train()

    losses_m = utils.AverageMeter()

    pbar = progress_bar(loader, parent=mbar, leave=False)
    pbar.update(0)

    for batch_idx, (input, target) in enumerate(loader):
        input, target = input.cuda(), target.cuda()
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(input)
            loss = loss_fn(output, target)
            
        losses_m.update(loss.item(), input.size(0))

        grad_scaler.scale(loss).backward()
        grad_scaler.step(optimizer)
        grad_scaler.update()

        if lr_scheduler is not None:
            lr_scheduler.step()

        pbar.update(batch_idx + 1)
        pbar.comment = f"{losses_m.avg:.4f}"

    pbar.on_iter_end()
    return OrderedDict([("loss", losses_m.avg)])


@torch.inference_mode()
def validate(model: nn.Module, loader: Iterable, loss_fn: Callable, mbar: master_bar):
    model.eval()

    accs_m = utils.AverageMeter()
    losses_m = utils.AverageMeter()

    pbar = progress_bar(loader, parent=mbar, leave=False)
    pbar.update(0)

    for batch_idx, (input, target) in enumerate(loader):
        
        input, target = input.cuda(), target.cuda()
        output = torch.round(model(input))

        loss = loss_fn(output, target).item()
        losses_m.update(loss, input.size(0))

        acc = accuracy_multi(output, target).item()
        accs_m.update(acc, output.size(0))

        pbar.update(batch_idx + 1)

    pbar.on_iter_end()
    return OrderedDict([("loss", losses_m.avg), ("accuracy", accs_m.avg)])


### Run!

In [ ]:
train_new = train_new[~train_new['fold'].isna()].reset_index(drop=True)
train_new['fold'] = train_new['fold'].astype(int)
print(train_new['fold'].unique())

In [ ]:
def training_loop(fold):
    
#     if fold>0:
#         time.sleep(5*60)
    
    with IPyExperimentsPytorch(exp_enable=False, cl_set_seed=42, cl_compact=True):
        print()
        print("*" * 100)
        print(f"Training fold {fold}")
        print("*" * 100)

        torch.backends.cudnn.benchmark = True

        dataset_train = WadhwaniDataset(train_new.query("fold!=@fold").reset_index(drop=True), augs=TRAIN_AUG, mode="train")
        dataset_valid = WadhwaniDataset(train_new.query("fold==@fold").reset_index(drop=True), augs=VALID_AUG, mode="valid")

        print(f"TRAIN: {len(dataset_train)} | VALID: {len(dataset_valid)}")

        loader_train = torch.utils.data.DataLoader(dataset_train, CFG.BS, num_workers=8, shuffle=True, drop_last=True)
        loader_valid = torch.utils.data.DataLoader(dataset_valid, CFG.BS * 2, num_workers=8, shuffle=False)

        model = get_wadhwani_classification_model(CFG.MODEL, pretrained=True, drop_path_rate=0.15)
        model.cuda()
        # adamw
        optimizer = create_optimizer_v2(model, "lookahead_RAdam", lr=CFG.LR, weight_decay=CFG.WD, layer_decay=0.9)

        num_train_steps = len(loader_train) * CFG.EP
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_train_steps)

        train_loss_fn = nn.BCEWithLogitsLoss()
        valid_loss_fn = nn.BCEWithLogitsLoss()

        grad_scaler = torch.cuda.amp.GradScaler()

        print(f"Scheduled epochs: {CFG.EP}")

        mbar = master_bar(list(range(CFG.EP)))
        best_epoch, best_metric = 0, 0
        metric_names = ["epoch", "train_loss", "valid_loss", "accuracy", "time"]
        mbar.write([f"{l:.6f}" if isinstance(l, float) else str(l) for l in metric_names], table=True)

        for epoch in range(CFG.EP):
            start_time = time.time()
            mbar.update(epoch)

            train_metrics = train_one_epoch(
                model, loader_train, train_loss_fn, optimizer,
                lr_scheduler=lr_scheduler, mixup_fn=None, grad_scaler=grad_scaler, mbar=mbar)

            valid_metrics = validate(model, loader_valid, valid_loss_fn, mbar=mbar)
            
            elapsed = format_time(time.time() - start_time)
            epoch_log = [epoch,train_metrics["loss"], valid_metrics["loss"], valid_metrics["accuracy"], elapsed]
            mbar.write([f"{l:.6f}" if isinstance(l, float) else str(l) for l in epoch_log], table=True)

            if valid_metrics["accuracy"] > best_metric:
                best_epoch, best_metric = epoch, valid_metrics["accuracy"]
                dirpath = MODEL_FOLDER / (KERNEL_TYPE + f"_CV_{fold}.pth")
                if not CFG.debug:
                    torch.save(model.state_dict(), dirpath)

        mbar.on_iter_end()
        print("*** Best metric: {0} (epoch {1})".format(best_metric, best_epoch))

In [ ]:
for fold_idx in range(CFG.n_splits):
    training_loop(fold_idx)

### Fin 